## Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

In [2]:

import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from bs4 import BeautifulSoup

In [3]:
#scraping wikipedia webpage 
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(page.content, 'html.parser')
table = soup.find('tbody')

records = table.select('tr')

row = [r.get_text() for r in records]

In [4]:
#Pre-processing the data
df = pd.DataFrame(row)
df_col = df[0].str.split('\n', expand=True)
df_clean1 = df_col.rename(columns=df_col.iloc[0])

In [5]:
df_clean = df_clean1.drop(df_clean1.index[0])
df_clean.head()

,,Postcode,Borough,Neighbourhood,
1,,M1A,Not assigned,Not assigned,
2,,M2A,Not assigned,Not assigned,
3,,M3A,North York,Parkwoods,
4,,M4A,North York,Victoria Village,
5,,M5A,Downtown Toronto,Harbourfront,


In [6]:
df_wNA = df_clean[df_clean.Borough != 'Not assigned']
df_wNA.head()

,,Postcode,Borough,Neighbourhood,
3,,M3A,North York,Parkwoods,
4,,M4A,North York,Victoria Village,
5,,M5A,Downtown Toronto,Harbourfront,
6,,M5A,Downtown Toronto,Regent Park,
7,,M6A,North York,Lawrence Heights,


In [7]:
df_PC = df_wNA.groupby(['Postcode', 'Borough'], sort = False).agg(','.join)
df_PC.reset_index(inplace = True)
df_PC.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Not assigned


In [8]:
#Not assigned value of the Borough and the Neighborhood columns will be Queen's Park
df_QP = df_PC.replace("Not assigned", "Queen's Park")
df_QP.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [9]:
#use the .shape method to print the number of rows of your dataframe.
df_QP.shape


(103, 3)

In [15]:
df_QP.sort_values(by=['PostalCode'])

KeyError: 'PostalCode'

## Importing Postal codes coordinates

In [16]:
latnlong = pd.read_csv('https://cocl.us/Geospatial_data')
latnlong.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
latnlong.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
import pandas as pd

In [18]:
df_QP = pd.read_csv('list_of_postal_codes_of_canada.csv')
df_QP.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [19]:
merged_df = pd.merge(df_QP, latnlong, on='PostalCode')
merged_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [21]:
merged_df.to_csv('deospatial_data.csv', index=False)